In [1]:
%autoreload 2

In [2]:
from collections import defaultdict
from argparse import Namespace
import copy
import gzip
import itertools
import os
import pickle
import sys
import typing

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tatsu
import tatsu.ast
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
import tqdm
import sklearn
from sklearn.model_selection import GridSearchCV, train_test_split, KFold
from sklearn.pipeline import Pipeline

sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src'))
from src.ast_utils import _extract_game_id
from src import fitness_energy_utils as utils
from src.fitness_energy_utils import NON_FEATURE_COLUMNS
from src.fitness_features import *
from src.ast_counter_sampler import *
from src.ast_mcmc_regrowth import *

In [3]:
grammar = open('../dsl/dsl.ebnf').read()
grammar_parser = tatsu.compile(grammar)
game_asts = list(cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', grammar_parser, False, relative_path='..'))
real_game_texts = [ast_printer.ast_to_string(ast, '\n') for ast in game_asts]
regrown_game_texts = list(load_games_from_file('../dsl/ast-real-regrowth-samples.pddl'))
regrown_game_1024_texts = list(load_games_from_file('../dsl/ast-real-regrowth-samples-1024.pddl'))
print(len(real_game_texts), len(regrown_game_texts), len(regrown_game_texts) / 98, len(regrown_game_1024_texts), len(regrown_game_1024_texts) / 98)


98 6272 64.0 100352 1024.0


In [4]:
fitness_df = utils.load_fitness_data('../data/fitness_features_1024_regrowths.csv.gz')
print(fitness_df.src_file.unique())
fitness_df.head()

['interactive-beta.pddl' 'ast-real-regrowth-samples-1024.pddl']


,Index,src_file,game_name,domain_name,original_game_name,real,all_variables_defined,all_variables_used,all_preferences_used,setup_objects_used,...,ast_ngram_constraints_n_6_score,ast_ngram_constraints_n_7_score,ast_ngram_terminal_n_2_score,ast_ngram_terminal_n_3_score,ast_ngram_terminal_n_4_score,ast_ngram_terminal_n_5_score,ast_ngram_scoring_n_2_score,ast_ngram_scoring_n_3_score,ast_ngram_scoring_n_4_score,ast_ngram_scoring_n_5_score
0,0,interactive-beta.pddl,6172feb1665491d1efbce164-0,medium-objects-room-v1,6172feb1665491d1efbce164-0,1,1,1,1,1.00,...,0.976714,0.984956,0.942266,0.970180,0.973574,0.978002,0.725160,0.761936,0.942521,0.975955
1,1,interactive-beta.pddl,5f77754ba932fb2c4ba181d8-2,many-objects-room-v1,5f77754ba932fb2c4ba181d8-2,1,1,1,1,1.00,...,0.986443,0.986721,0.935275,0.965656,0.967074,0.978883,0.670913,0.801850,0.870098,0.924083
2,2,interactive-beta.pddl,614b603d4da88384282967a7-3,many-objects-room-v1,614b603d4da88384282967a7-3,1,1,1,1,0.00,...,0.976849,0.981247,0.000000,0.000000,0.000000,0.000000,0.452633,0.593800,0.814709,0.883131
3,3,interactive-beta.pddl,5bc79f652885710001a0e82a-5,few-objects-room-v1,5bc79f652885710001a0e82a-5,1,1,1,1,0.00,...,0.986600,0.986476,0.000000,0.000000,0.000000,0.000000,0.725160,0.761936,0.942521,0.975955
4,4,interactive-beta.pddl,614dec67f6eb129c3a77defd-6,medium-objects-room-v1,614dec67f6eb129c3a77defd-6,1,1,1,1,0.25,...,0.985217,0.983768,0.000000,0.000000,0.000000,0.000000,0.937248,0.947704,0.933456,0.976437


# Plan of attack
* Use the code in `ast_counter_sampler.py` to generate a sample from the MLE
* Score it with a fitness function adapted to working with a single example at a time
* At each subsequent iteration, regrow the game from a random node, and score the regrowth
* Since we have energy scores before and after, we can either accept greedily if $E_{new} < E_{old}$, or accept with probability $\exp(\beta (E_{old} - E_{new} ))$

In [5]:
DEFAULT_ARGS = argparse.Namespace(
    grammar_file=os.path.join('..', DEFAULT_GRAMMAR_FILE),
    parse_counter=False,
    counter_output_path=os.path.join('..', DEFAULT_COUNTER_OUTPUT_PATH),
    random_seed=DEFUALT_RANDOM_SEED,
)

In [6]:
FITNESS_MODEL_DATE_ID = 'full_features_2023_03_28'
FITNESS_FEATURIZER_PATH = '../models/fitness_featurizer_2023_03_28.pkl.gz'


mcmc = MCMCRegrowthSampler(DEFAULT_ARGS,
    FITNESS_MODEL_DATE_ID, FITNESS_FEATURIZER_PATH, greedy_acceptance=True, 
    plateau_patience_steps=1000, max_steps=20000,
    fitness_function_relative_path='..')  #   acceptance_temperature=10.0, 

In [7]:
n_samples = 10
mcmc.multiple_samples(n_samples, verbose=1, should_tqdm=True)

  0%|          | 0/10 [00:00<?, ?it/s]

Plateaued at step 3055 with energy -59.95831
Plateaued at step 3082 with energy -64.96738
Plateaued at step 6918 with energy -66.61777
Plateaued at step 8497 with energy -54.97644
Plateaued at step 2549 with energy -66.70599
Plateaued at step 8181 with energy -67.38096
Plateaued at step 10663 with energy -59.18020
Plateaued at step 9867 with energy -70.90728
Plateaued at step 6470 with energy -70.10515


In [ ]:
from argparse import Namespace
from src.fitness_features import *
args = Namespace(no_binarize=False, no_merge=True)
featurizer = build_fitness_featurizer(args)
r = featurizer.parse(mcmc.samples[4][0], '', return_row=True, preprocess_row=False)

## Regrwoth from existing games

In [9]:
n_samples = 5
n_games = 10
start = 25
end = start + n_games


for original_index in range(start, end):
    print(f'Generating samples starting from original index {original_index} (id {_extract_game_id(real_game_texts[original_index])})')
    mcmc.multiple_samples(n_samples, verbose=1, should_tqdm=False, initial_proposal=game_asts[original_index], postprocess=False)

    # print()
    # for i in range(n_samples * (original_index - start), n_samples * (original_index - start + 1)):
    #     print(i, mcmc.samples[i][2])

Generating samples starting from original index 25 (id 56cb8858edf8da000b6df354-32)
Plateaued at step 1397 with energy -76.33878 (initial proposal energy: -75.47111)
Plateaued at step 1756 with energy -77.10758 (initial proposal energy: -75.47111)
Plateaued at step 1672 with energy -77.95261 (initial proposal energy: -75.47111)
Plateaued at step 1759 with energy -77.75713 (initial proposal energy: -75.47111)
Plateaued at step 1338 with energy -77.63092 (initial proposal energy: -75.47111)
Generating samples starting from original index 26 (id 614e1599db14d8f3a5c1486a-33)
Plateaued at step 1001 with energy -79.68496 (initial proposal energy: -79.68496)
Plateaued at step 1001 with energy -79.68496 (initial proposal energy: -79.68496)
Plateaued at step 1001 with energy -79.68496 (initial proposal energy: -79.68496)
Plateaued at step 1001 with energy -79.68496 (initial proposal energy: -79.68496)
Plateaued at step 1001 with energy -79.68496 (initial proposal energy: -79.68496)
Generating s

In [10]:
len(mcmc.samples)

85

In [42]:
mcmc.visualize_sample(11)

### Energy of real game: -76.363 | Energy of regrown game: -77.092 | Difference: -0.729

### Top features changing the game's energy
feature name: **value** = (original feature value => regrown feature value) * weight

| Features increasing energy (= more fake)                            | Features decreasing energy (= more real)                                                         |
|---------------------------------------------------------------------|--------------------------------------------------------------------------------------------------|
| ast_ngram_full_n_7_score: **1.504** = (0.981 => 0.929) * -28.958    | pref_forall_count_once_per_external_objects_used_correct: **-0.805** = (0.000 => 1.000) * -0.805 |
| ast_ngram_terminal_n_5_score: **0.084** = (0.958 => 0.027) * -0.090 | node_count_terminal_2: **-0.777** = (1.000 => 0.000) * 0.777                                     |
| mean_depth_scoring_3: **0.042** = (0.000 => 1.000) * 0.042          | mean_depth_scoring_4: **-0.564** = (1.000 => 0.000) * 0.564                                      |
|                                                                     | ast_ngram_scoring_n_5_score: **-0.142** = (1.000 => 0.954) * 3.063                               |
|                                                                     | node_count_terminal_1: **-0.072** = (0.000 => 1.000) * -0.072                                    |

### Game Diffs

f,1,(define (game 5996d2256b939900012d9f22-24) (:domain few-objects-room-v1),f,1,(define (game 5996d2256b939900012d9f22-24) (:domain few-objects-room-v1)
,2,(:setup,,2,(:setup
,3,(and,,3,(and
,4,(exists (?c - chair ?h - hexagonal_bin),,4,(exists (?c - chair ?h - hexagonal_bin)
,5,(game-conserved,,5,(game-conserved
,6,(on ?c ?h),,6,(on ?c ?h)
,7,),,7,)
,8,),,8,)
,9,),,9,)
,10,),,10,)
,11,(:constraints,,11,(:constraints


### Feature Diffs

node_count_terminal_2: -1.000 (1.000 => 0.000)
mean_depth_scoring_4: -1.000 (1.000 => 0.000)
ast_ngram_terminal_n_5_score: -0.931 (0.958 => 0.027)
ast_ngram_full_n_7_score: -0.052 (0.981 => 0.929)
ast_ngram_scoring_n_5_score: -0.046 (1.000 => 0.954)
node_count_terminal_1: 1.000 (0.000 => 1.000)
mean_depth_scoring_3: 1.000 (0.000 => 1.000)
pref_forall_count_once_per_external_objects_used_correct: 1.000 (0.000 => 1.000)


In [72]:
mcmc.samples[75][0][4][1].preferences.definition.forall_pref.preferences.pref_body.body.exists_args.then_funcs[0].seq_func.once_pred.pred.and_args[-1].pred

{'comp': {'comp_op': '>', 'arg_1': {'arg': {'func': {'arg_1': {'term': '?w1', 'parseinfo': {'tokenizer': None, 'rule': 'predicate_or_function_location_term', 'pos': 812, 'endpos': 816, 'line': 22, 'endline': 22, 'alerts': []}}, 'arg_2': {'term': '?g', 'parseinfo': {'tokenizer': None, 'rule': 'predicate_or_function_term', 'pos': 816, 'endpos': 819, 'line': 22, 'endline': 22, 'alerts': []}}, 'parseinfo': {'tokenizer': None, 'rule': 'function_distance', 'pos': 803, 'endpos': 820, 'line': 22, 'endline': 22, 'alerts': []}}, 'parseinfo': {'tokenizer': None, 'rule': 'function_eval', 'pos': 803, 'endpos': 820, 'line': 22, 'endline': 22, 'alerts': []}}, 'parseinfo': {'tokenizer': None, 'rule': 'comparison_arg', 'pos': 802, 'endpos': 820, 'line': 22, 'endline': 22, 'alerts': []}}, 'arg_2': {'arg': {'func': {'arg_1': {'term': '?w2', 'parseinfo': {'tokenizer': None, 'rule': 'predicate_or_function_location_term', 'pos': 830, 'endpos': 834, 'line': 22, 'endline': 22, 'alerts': []}}, 'arg_2': {'term'

In [80]:
mcmc.samples[11][0][5][1]

{'terminal': {'op': '>=', 'expr_1': {'expr': '(total-score)', 'parseinfo': {'tokenizer': None, 'rule': 'terminal_expr', 'pos': 618, 'endpos': 632, 'line': 18, 'endline': 18, 'alerts': []}}, 'expr_2': {'expr': '(total-score)', 'parseinfo': {'tokenizer': None, 'rule': 'terminal_expr', 'pos': 116146, 'endpos': 116146, 'line': 116146, 'endline': 116146, 'alerts': []}}, 'parseinfo': {'tokenizer': None, 'rule': 'terminal_comp', 'pos': 615, 'endpos': 637, 'line': 18, 'endline': 18, 'alerts': []}}, 'parseinfo': {'tokenizer': None, 'rule': 'terminal', 'pos': 610, 'endpos': 637, 'line': 17, 'endline': 18, 'alerts': []}}

In [85]:
from ast_parser import ASTBooleanParser
bp = ASTBooleanParser()
bp.evaluate_tautology(bp(mcmc.samples[11][0][5][1], section='(:terminal'))
# bp(mcmc.samples[75][0][4][1].preferences.definition.forall_pref.preferences.pref_body.body.exists_args.then_funcs[0].seq_func.once_pred.pred.and_args[-1].pred, section='(:constraints')

True

In [94]:
import ast_utils
terminal = copy.deepcopy(mcmc.samples[11][3][5][1])
ast_utils.replace_child(terminal.terminal, 'expr_1', '400')
bp.game_start()
bp(terminal, section='(:terminal')

TRUE

In [39]:
from argparse import Namespace
from src.fitness_features import *
args = Namespace(no_binarize=False, no_merge=False)
featurizer = build_fitness_featurizer(args)

In [41]:
mcmc.fitness_featurizer.parse(mcmc.samples[50][0], 'mcmc', True, True)['ast_ngram_scoring_n_5_score']

-1.1492754355700883

In [ ]:
ps = {k:v for k, v in featurizer.parse(mcmc.samples[37][0], 'mcmc', True, False).items() if 'all_variables' in k}
ps

In [ ]:
# %prun -r 
n_samples = 5
start = 0
end = start + n_samples

for original_index in range(start, end):
    print(f'Generating samples starting from original index {original_index} (id {_extract_game_id(real_game_texts[original_index])})')
    mcmc.multiple_samples(n_samples, verbose=1, should_tqdm=True, initial_proposal=game_asts[original_index])

    print()
    for i in range(n_samples * (original_index - start), n_samples * (original_index - start + 1)):
        print(i, mcmc.samples[i][2])

In [ ]:
mcmc.visualize_sample(14)

In [ ]:
mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1].score(ast_printer.ast_to_string(mcmc.samples[14][0], ' '))

In [ ]:
mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1].score(ast_printer.ast_to_string(mcmc.samples[14][3], ' '))

In [ ]:
ngram_model = mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1]

sample_ngrams = [('constraints', 'and', 'forall', 'wall', 'and'), ('and', 'forall', 'wall', 'and', 'preference'), ('forall', 'wall', 'and', 'preference', 'preferenceName'), ('wall', 'and', 'preference', 'preferenceName', 'exists'), ('and', 'preference', 'preferenceName', 'exists', 'block'), ('preference', 'preferenceName', 'exists', 'block', 'atend'), ('preferenceName', 'exists', 'block', 'atend', 'in'), ('exists', 'block', 'atend', 'in', 'scoring'), ('block', 'atend', 'in', 'scoring', 'externalforallmaximize'), ('atend', 'in', 'scoring', 'externalforallmaximize', 'countonceperobjects'), ('in', 'scoring', 'externalforallmaximize', 'countonceperobjects', 'preferenceName')]
original_ngrams = [('constraints', 'and', 'forall', 'building', 'and'), ('and', 'forall', 'building', 'and', 'preference'), ('forall', 'building', 'and', 'preference', 'preferenceName'), ('building', 'and', 'preference', 'preferenceName', 'exists'), ('and', 'preference', 'preferenceName', 'exists', 'block'), ('preference', 'preferenceName', 'exists', 'block', 'atend'), ('preferenceName', 'exists', 'block', 'atend', 'in'), ('exists', 'block', 'atend', 'in', 'scoring'), ('block', 'atend', 'in', 'scoring', 'externalforallmaximize'), ('atend', 'in', 'scoring', 'externalforallmaximize', 'countonceperobjects'), ('in', 'scoring', 'externalforallmaximize', 'countonceperobjects', 'preferenceName')]

print(f'Sample ngrams: {np.mean([ngram_model.ngram_logprobs[s] for s in sample_ngrams])} | ')
for s in sample_ngrams:
    print(s, ngram_model.ngram_counts[s], ngram_model.ngram_logprobs[s])

print()
print(f'Original ngrams: {np.mean([ngram_model.ngram_logprobs[s] for s in original_ngrams])}')
for s in original_ngrams:
    print(s, ngram_model.ngram_counts[s], ngram_model.ngram_logprobs[s])
print()

for s in sample_ngrams[:]:
    if s in original_ngrams:
        sample_ngrams.remove(s)
        original_ngrams.remove(s)

print('Sample remaining ngrams')
for s in sample_ngrams:
    print(s, ngram_model.ngram_counts[s], ngram_model.ngram_logprobs[s])

print()
print('Original remaining ngrams')
for s in original_ngrams:
    print(s, ngram_model.ngram_counts[s], ngram_model.ngram_logprobs[s])

In [ ]:
original_ngrams = [('constraints', 'and', 'forall', 'building', 'and'), ('and', 'forall', 'building', 'and', 'preference'), ('forall', 'building', 'and', 'preference', 'preferenceName'), ('building', 'and', 'preference', 'preferenceName', 'exists'), ('and', 'preference', 'preferenceName', 'exists', 'block'), ('preference', 'preferenceName', 'exists', 'block', 'atend'), ('preferenceName', 'exists', 'block', 'atend', 'in'), ('exists', 'block', 'atend', 'in', 'scoring'), ('block', 'atend', 'in', 'scoring', 'externalforallmaximize'), ('atend', 'in', 'scoring', 'externalforallmaximize', 'countonceperobjects'), ('in', 'scoring', 'externalforallmaximize', 'countonceperobjects', 'preferenceName')]
print('Original ngrams')
for s in original_ngrams:
    print(s, mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1].ngram_counts[s], mcmc.fitness_featurizer.full_text_registry[0].n_gram_model.models[-1].ngram_logprobs[s])

In [ ]:
mcmc.visualize_sample(7)

In [ ]:
mcmc.visualize_sample(15)

### Notes from looking at these games
* **TODO:** debug the 'section-without' logic, it seems to not work
* **TODO:** add features for preferences with different lengths of modals

In [ ]:
print(ast_printer.ast_to_string(samples[8][0], '\n'))

In [ ]:
fitness_featurizer = build_fitness_featurizer(DEFAULT_ARGS)
features = fitness_featurizer.parse(samples[7][0], 'mcmc', True)
{k: v for k, v in features.items() if 'correctly' in k or 'forall' in k or 'no' in k}

In [ ]:
with gzip.open(DEFAULT_FITNESS_FUNCTION_PATH, 'rb') as f:
    cv_fitness_model = pickle.load(f)

In [ ]:
weights = cv_fitness_model.named_steps['fitness'].model.fc1.weight.data.detach().numpy().squeeze()
bias = cv_fitness_model.named_steps['fitness'].model.fc1.bias.data.detach().numpy().squeeze()
print(weights.mean(), bias)

In [ ]:
plt.hist(weights, bins=100)
plt.title('Fitness Model Weights')
plt.xlabel('Weight magnitude')
plt.ylabel('Count')

In [ ]:
import ast_crossover_sampler
asts = [ast for ast in cached_load_and_parse_games_from_file('../dsl/interactive-beta.pddl', 
    mcmc.grammar_parser, False, relative_path='..')]

mcmc_with_crossover = MCMCRegrowthCrossoverSampler(DEFAULT_ARGS, 
    ast_crossover_sampler.CrossoverType.SAME_RULE, asts, 0.25,
    '../models/cv_fitness_model_2022_12_26.pkl.gz', 
    greedy_acceptance=True, plateau_patience_steps=20, acceptance_temperature=10.0, max_steps=1000)

In [ ]:
mcmc_with_crossover.multiple_samples(10, verbose=2, should_tqdm=True)
print([x[2] for x in mcmc_with_crossover.samples])

In [ ]:
print([x[2] for x in mcmc_with_crossover.samples])